<a href="https://colab.research.google.com/github/GabaeOh/study_data_analytics/blob/main/codes/NLP/healthapp_review_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

refer : !pip install python-mecab-ko pyLDAvis

In [1]:
#!pip install python-mecab-ko pyLDAvis

In [2]:
import numpy as np

In [3]:
import pandas as pd
df_review_negative = pd.read_csv('./review_negative.csv')
df_review_positive = pd.read_csv('./review_positive.csv')

In [4]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        413 non-null    int64 
 1   app               413 non-null    object
 2   review            413 non-null    object
 3   rating            413 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 19.5+ KB


In [5]:
df_review_negative = df_review_negative.dropna()

## LDA 분석

#### 문장 벡터화

In [28]:
from gensim.corpora import Dictionary

In [29]:
dictionary = Dictionary(df_review_negative[['replaced_review']].values)
dictionary

In [30]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3])

[(3, 1)]

In [31]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [32]:
df_review_negative['문장벡터화'] = corpus_list

In [33]:
df_review_negative.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장벡터화'],
      dtype='object')

#### 토픽 잡기

In [34]:
from gensim.models import LdaModel

In [35]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [36]:
lda_model.print_topics(num_words=4)

[(0, '0.010*"광고" + 0.009*"리뷰" + 0.009*"동영상" + 0.006*"수정 부탁"'),
 (1, '0.028*"환불" + 0.026*"로그인" + 0.024*"운동" + 0.013*"불편"'),
 (2, '0.010*"광고 짜증" + 0.009*"위젯" + 0.009*"최악" + 0.009*"의견"')]

#### 토픽 분석
-  review_positive.csv 토픽 분석
- 토픽 수는 3으로 가정

In [15]:
# 'replaced_review' 열의 모든 값을 문자열로 변환
df_review_positive['replaced_review'] = df_review_positive['replaced_review'].astype(str)

In [16]:
dictionary = Dictionary(df_review_positive[['replaced_review']].values)
dictionary

In [17]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_positive[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [18]:
df_review_positive['문장벡터화'] = corpus_list

In [19]:
df_review_positive[:3]

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review,문장벡터화
0,1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,NaN,nan,"[(0, 1)]"
1,1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,최고,"[(1, 1)]"
2,1002,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무좋아요,5,NaN,nan,"[(0, 1)]"


In [20]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [21]:
lda_model.print_topics(num_words=4)

[(0, '0.024*"뱃살" + 0.021*"감사" + 0.021*"도움" + 0.019*"광고"'),
 (1, '0.014*"추천" + 0.013*"운동 최고" + 0.010*"nan" + 0.010*"강추"'),
 (2, '0.623*"nan" + 0.111*"운동" + 0.036*"최고" + 0.006*"운동 운동"')]

#### LDA 시각화

In [ ]:
%pip install pyLDAvis

In [37]:
import pyLDAvis.gensim_models

In [38]:
pyLDAvis.enable_notebook() # 일반적인 python에선 불필요
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)

In [40]:
pyLDAvis.display(result_visualized)

In [41]:
pyLDAvis.save_html(result_visualized, './result_visualized.html')